In [1]:
!pip install --upgrade transformers datasets accelerate peft

  Using cached datasets-3.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 17.6 MB/s eta 0:00:00a 0:00:01
Using cached datasets-3.0.0-py3-none-any.whl (474 kB)
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Attempting uninstall: datasets
    Found existing installation: datasets

In [2]:
# This step is needed only on Apple Metal
!pip uninstall bitsandbytes -y

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # Replace with the appropriate Llama 3.1 model name

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up LoRA configuration
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout probability
    bias="none",  # Don't add bias to the LoRA adapters
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to specific attention layers
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Check if pad token exists, add it if missing
if tokenizer.pad_token is None:
   print(f'Added pad_token {tokenizer.eos_token}')
   tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

# Update the model's token embeddings to accommodate the new pad token
model.resize_token_embeddings(len(tokenizer))

Added pad_token <|eot_id|>


Embedding(128256, 4096)

In [5]:
import torch
print(f'MPS available: {torch.backends.mps.is_available()}')

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = model.to(device)

MPS available: True


Prepare data

In [6]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot", split='all')
dataset

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 256916
})

In [7]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


In [8]:
from copy import deepcopy

IGNORE_INDEX = -100  # The default setting in CrossEntropyLoss

# Tokenize the dataset
def tokenize_function(example):

    if example.get("Patient", "") == "":
        prompt = PROMPT_DICT["prompt_no_input"].format(instruction=example['Description'])
    else:
        prompt = PROMPT_DICT["prompt_input"].format(instruction=example['Description'], input=example['Patient'])

    formatted_example = prompt + example['Doctor']

    encoded_prompt = torch.tensor(
            tokenizer.encode(prompt), dtype=torch.int64
        )
        
    example = tokenizer.encode(formatted_example)
    example.append(tokenizer.eos_token_id)
    example = torch.tensor(
        example, dtype=torch.int64
    )
    labels = deepcopy(example)
    labels[: len(encoded_prompt)] = -1
    example_mask = example.ge(0)
    label_mask = labels.ge(0)
    example[~example_mask] = 0
    labels[~label_mask] = IGNORE_INDEX

    return {
        "input_ids": example.tolist(),
        "labels": labels.tolist(),
        "attention_mask": example_mask.tolist(),
    }


tokenized_dataset = dataset.map(tokenize_function, remove_columns=['Description', 'Patient', 'Doctor'], batched=False)
tokenized_dataset

Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 256916
})

In [9]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

Training

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [16]:
from transformers import TrainingArguments, Trainer
from torch.optim import AdamW

training_args = TrainingArguments(
    output_dir="./llama_lora_finetuned",
    eval_strategy="steps",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    save_steps=1000,
    logging_steps=50,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=10,
    load_best_model_at_end=True,
)

optimizer = AdamW(model.parameters(), lr=5e-5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset[:100],
    data_collator=data_collator,
    optimizers=(optimizer, None),  # No need for a scheduler here
)


In [17]:
trainer.train()

  0%|          | 0/205532 [00:00<?, ?it/s]

{'loss': 2.883, 'grad_norm': 3335.911376953125, 'learning_rate': 4.999026868169831e-05, 'epoch': 0.0}


  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_input_ids_runtime': 261.0656, 'eval_input_ids_samples_per_second': 0.383, 'eval_input_ids_steps_per_second': 0.383, 'epoch': 0.0}


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_labels_runtime': 90.4632, 'eval_labels_samples_per_second': 1.105, 'eval_labels_steps_per_second': 1.105, 'epoch': 0.0}


  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_attention_mask_runtime': 137.9367, 'eval_attention_mask_samples_per_second': 0.725, 'eval_attention_mask_steps_per_second': 0.725, 'epoch': 0.0}
{'loss': 5.772, 'grad_norm': 19.55955696105957, 'learning_rate': 4.996594038594409e-05, 'epoch': 0.0}


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained("./llama3_lora_model_finetuned")
tokenizer.save_pretrained("./llama3_lora_tokenizer_finetuned")